## 第8章 文埋め込み

### 文埋め込みモデルの実装

#### 教師なしSimCSEの実装

In [1]:
from transformers.trainer_utils import set_seed

# 乱数シードの設定
set_seed(42)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### データセットの読み込みと前処理

In [2]:
from datasets import load_dataset

unsup_train_dataset = load_dataset(
    "llm-book/jawiki-sentences", split="train"
)

Using custom data configuration default
Reusing dataset jawiki-sentences (/root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc)


In [3]:
# 訓練セットの形式と事例数を確認
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 24387500
})


In [4]:
# 訓練セットの中身を確認する
for i, text in enumerate(unsup_train_dataset[:50]["text"]):
    print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。
5 "&" はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。
6 結果として、アルファベットの復唱の最後は "X, Y, Z, and per se and" という形になった。
7 この最後のフレーズが繰り返されるうちに "ampersand" と訛っていき、この言葉は1837年までには英語の一般的な語法となった。
8 アンドレ=マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が "Ampère's and" と呼ばれるようになったという誤った語源俗説がある。
9 アンパサンドの起源は1世紀の古ローマ筆記体にまで遡ることができる。
10 古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた(左図「アンパサンドの変遷」の字形1)。それに続く、流麗さを増した新ローマ筆記体では、様々な合字が極めて頻繁に使われるようになった。
11 字形2と3は4世紀中頃における et の合字の例である。
12 その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れていった。
13 しかし、et の合字は使われ続け、次第に元の文字がわかりにくい字形に変化していった(字形4から6)。
14 現代のイタリック体のアンパサンドは、ルネサンス期に発展した筆記体での et の合字に遡る。
15 1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアンパサンドの両方を多用するようになった。
16

In [5]:
# 訓練セットから空白行を事例を削除
unsup_train_dataset = unsup_train_dataset.filter(
    lambda example: example["text"].strip() != ""
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc/cache-ecc223a34f1ffe88.arrow


In [6]:
# 訓練セットをシャッフルし、最初の100万事例を取り出す
unsup_train_dataset = unsup_train_dataset.shuffle().select(
    range(1000000)
)
# パフォーマンスの低下を防ぐために、シャッフルされた状態の訓練セットを
# ディスクに書き込む
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc/cache-e9e6caaca5ef995f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/53a30ee0f53283c9671cc04dc79a18905ce320760396d0e87085fcd63cbfa3fc/cache-9484a91dba79bc01.arrow


In [7]:
# 前処理後の訓練セットの形式と事例数を確認
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


In [8]:
# 前処理後の訓練セットの内容を確認
for i, text in enumerate(unsup_train_dataset[:10]["text"]):
    print(i, text)

0 2005年の時点で、10,000人ものウズベキスタン人が韓国での労働に従事しており、その大部分が高麗人である。
1 小学5年生(11歳)の時から芸能活動を開始。
2 i ħ d d t | ψ ( t ) ⟩ = L ^ | ψ ( t ) ⟩ {\displaystyle i\hbar {\frac {d}{dt}}|\psi (t)\rangle ={\hat {L}}|\psi (t)\rangle }
3 安土宗論(あづちしゅうろん)は、1579年(天正7年)、安土城下の浄厳院で行われた浄土宗と法華宗の宗論。
4 1927年 オーストラリア選手権(1927ねんオーストラリアせんしゅけん、1927 Australian Championships)に関する記事。
5 さらにマップ上で最大8つまでしか建築できず(司令官アビリティの”解体”か設置したプレイヤー自らが出向いて解体する必要がある)
6 特に誉淳が1827年から作成した『古瓦譜』は畿内で600点以上の拓本を蒐集し、瓦当文様に着目したうえで編年を試みている。
7 マルクス主義者を広言し、メキシコ共産党の敵であり味方であった。
8 ICHILLIN'(アイチリン、朝: 아이칠린)は、韓国の7人組女性アイドルグループ。
9 マークVIは1983年にモデルサイクルを終了し、1984年のマークVII(英語版)はフルサイズセグメントから撤退し、マークシリーズは異なるセグメントに移行した。


In [9]:
# Hugging Face Hubのllm-book/JGLUEのリポジトリから
# JSTSデータセットの訓練セットと検証セットを読み込み、
# それぞれをSimCSEの検証セットとテストセットとして使用する
valid_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="train"
)
test_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="validation"
)

Reusing dataset jglue (/root/.cache/huggingface/datasets/llm-book___jglue/JSTS/1.1.0/b394a8dbefe82fb1dc2724c1eb79bb1ea3062df2037f91a69a27c089f3ff685f)
Reusing dataset jglue (/root/.cache/huggingface/datasets/llm-book___jglue/JSTS/1.1.0/b394a8dbefe82fb1dc2724c1eb79bb1ea3062df2037f91a69a27c089f3ff685f)


#### トークナイザとcollate関数の準備

In [10]:
from transformers import AutoTokenizer

base_model_name = "tohoku-nlp/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [11]:
import torch
from torch import Tensor
from transformers import BatchEncoding

def unsup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """教師なしSimCSEの訓練セットのミニバッチを作成"""
    # ミニバッチに含まれる文にトークナイザを適用する
    tokenized_texts = tokenizer(
        [example["text"] for example in examples],
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 文と文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文）に対してi列目の事例（文）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    return {
        "tokenized_texts_1": tokenized_texts,
        "tokenized_texts_2": tokenized_texts,
        "labels": labels,
    }

In [12]:
def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """SimCSEの検証・テストセットのミニバッチを作成"""
    # ミニバッチの文ペアに含まれる文（文1と文2）のそれぞれに
    # トークナイザを適用する
    tokenized_texts_1 = tokenizer(
        [example["sentence1"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized_texts_2 = tokenizer(
        [example["sentence2"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # 文1と文2の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文1）に対して
    # i列目の事例（文2）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    # データセットに付与された類似度スコアのTensorを作成する
    label_scores = torch.tensor(
        [example["label"] for example in examples]
    )

    return {
        "tokenized_texts_1": tokenized_texts_1,
        "tokenized_texts_2": tokenized_texts_2,
        "labels": labels,
        "label_scores": label_scores,
    }

#### モデル準備

In [13]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.utils import ModelOutput

class SimCSEModel(nn.Module):
    """SimCSEのモデル"""

    def __init__(
        self,
        base_model_name: str,
        mlp_only_train: bool = False,
        temperature: float = 0.05,
    ):
        """モデルの初期化"""
        super().__init__()

        # モデル名からエンコーダを初期化する
        self.encoder = AutoModel.from_pretrained(base_model_name)
        # パラメータをメモリ上に隣接した形で配置
        # これを実行しない場合、モデルの保存でエラーになることがある
        # for param in model.parameters():
        #     param.data = param.data.contiguous()
        # MLP層の次元数
        self.hidden_size = self.encoder.config.hidden_size
        # MLP層の線形層
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        # MLP層の活性化関数
        self.activation = nn.Tanh()

        # MLP層による変換を訓練時にのみ適用するよう設定するフラグ
        self.mlp_only_train = mlp_only_train
        # 交差エントロピー損失の計算時に使用する温度
        self.temperature = temperature

    def encode_texts(self, tokenized_texts: BatchEncoding) -> Tensor:
        """エンコーダを用いて文をベクトルに変換"""
        # トークナイズされた文をエンコーダに入力する
        encoded_texts = self.encoder(**tokenized_texts)
        # モデルの最終層の出力（last_hidden_state）の
        # [CLS]トークン（0番目の位置のトークン）のベクトルを取り出す
        encoded_texts = encoded_texts.last_hidden_state[:, 0]

        # self.mlp_only_trainのフラグがTrueに設定されていて
        # かつ訓練時でない場合、MLP層の変換を適用せずにベクトルを返す
        if self.mlp_only_train and not self.training:
            return encoded_texts

        # MLP層によるベクトルの変換を行う
        encoded_texts = self.dense(encoded_texts)
        encoded_texts = self.activation(encoded_texts)

        return encoded_texts

    def forward(
        self,
        tokenized_texts_1: BatchEncoding,
        tokenized_texts_2: BatchEncoding,
        labels: Tensor,
        label_scores: Tensor | None = None,
    ) -> ModelOutput:
        """モデルの前向き計算を定義"""
        # 文ペアをベクトルに変換する
        encoded_texts_1 = self.encode_texts(tokenized_texts_1)
        encoded_texts_2 = self.encode_texts(tokenized_texts_2)

        # 文ペアの類似度行列を作成する
        sim_matrix = F.cosine_similarity(
            encoded_texts_1.unsqueeze(1),
            encoded_texts_2.unsqueeze(0),
            dim=2,
        )

        # 交差エントロピー損失を求める
        loss = F.cross_entropy(sim_matrix / self.temperature, labels)

        # 性能評価に使用するため、正例ペアに対するスコアを類似度行列から取り出す
        positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
        positive_scores = torch.masked_select(
            sim_matrix, positive_mask
        )

        return ModelOutput(loss=loss, scores=positive_scores)

# 教師なしSimCSEのモデルを初期化する
unsup_model = SimCSEModel(base_model_name, mlp_only_train=True)

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

#### Trainerの準備

In [14]:
from scipy.stats import spearmanr
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction) -> dict[str, float]:
    """
    モデルが予測したスコアと評価用データのスコアの
    スピアマンの順位相関係数を計算
    """
    scores = p.predictions
    labels, label_scores = p.label_ids

    spearman = spearmanr(scores, label_scores).correlation

    return {"spearman": spearman}

In [15]:
from transformers import TrainingArguments

# 教師なしSimCSEの訓練のハイパーパラメータを設定する
unsup_training_args = TrainingArguments(
    output_dir="../model/outputs_unsup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=256,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=256,  # 評価時のバッチサイズ
    learning_rate=3e-5,  # 学習率
    num_train_epochs=1,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
    report_to="none",  # 外部ツールへのログを無効化
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# !pip install transformers[torch]

In [17]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import Trainer

class SimCSETrainer(Trainer):
    """SimCSEの訓練に使用するTrainer"""

    def get_eval_dataloader(
        self, eval_dataset: Dataset | None = None
    ) -> DataLoader:
        """
        検証・テストセットのDataLoaderでeval_collate_fnを使うように
        Trainerのget_eval_dataloaderをオーバーライド
        """
        if eval_dataset is None:
            eval_dataset = self.eval_dataset

        return DataLoader(
            eval_dataset,
            batch_size=64,
            collate_fn=eval_collate_fn,
            pin_memory=True,
        )

# 教師なしSimCSEのTrainerを初期化する
unsup_trainer = SimCSETrainer(
    model=unsup_model,
    args=unsup_training_args,
    data_collator=unsup_train_collate_fn,
    train_dataset=unsup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

####  訓練の実行

In [77]:
# 教師なしSimCSEの訓練を行う
unsup_trainer.train()

TrainOutput(global_step=3907, training_loss=0.00033567894401759603, metrics={'train_runtime': 1629.5441, 'train_samples_per_second': 613.669, 'train_steps_per_second': 2.398, 'total_flos': 0.0, 'train_loss': 0.00033567894401759603, 'epoch': 1.0})

#### 性能評価

In [78]:
# 検証セットで教師なしSimCSEのモデル評価を行う
unsup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.3203227519989014,
 'eval_spearman': 0.7608961210671525,
 'eval_runtime': 10.878,
 'eval_samples_per_second': 1144.601,
 'eval_steps_per_second': 4.504,
 'epoch': 1.0}

In [79]:
# テストセットで教師なしSimCSEのモデル評価を行う
unsup_trainer.evaluate(test_dataset)

{'eval_loss': 2.1774518489837646,
 'eval_spearman': 0.7878933714566898,
 'eval_runtime': 1.3524,
 'eval_samples_per_second': 1077.307,
 'eval_steps_per_second': 4.436,
 'epoch': 1.0}

In [80]:
#### トークナイザの保存とモデルの保存
encoder_path = "../model/outputs_unsup_simcse/encoder"
unsup_model.encoder.save_pretrained(encoder_path)
tokenizer.save_pretrained(encoder_path)

('../model/outputs_unsup_simcse/encoder/tokenizer_config.json',
 '../model/outputs_unsup_simcse/encoder/special_tokens_map.json',
 '../model/outputs_unsup_simcse/encoder/vocab.txt',
 '../model/outputs_unsup_simcse/encoder/added_tokens.json')

#### 教師ありSimCSEの実装

In [18]:
# 乱数シードの設定を行う
from transformers.trainer_utils import set_seed

set_seed(42)

#### データセットの読み込みと前処理

In [19]:
from datasets import load_dataset
# JSNLIの訓練セットを読み込む
jsnli_dataset = load_dataset("llm-book/jsnli", split="train")

Using custom data configuration default
Reusing dataset jsnli (/root/.cache/huggingface/datasets/llm-book___jsnli/default/1.0.0/b59ac9cb188ddb68dc451bae1b33ac9ebe501a5d2c41f17c5ec06ad0621186d7)


In [20]:
# JSNLIの訓練セットの形式と事例数を確認
print(jsnli_dataset)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 533005
})


In [21]:
from pprint import pprint

# JSNLIの訓練セットの内容を確認
pprint(jsnli_dataset[0])
pprint(jsnli_dataset[1])

{'hypothesis': '男 は 魔法 の ショー の ため に ナイフ を 投げる 行為 を 練習 して い ます 。',
 'label': 'neutral',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'hypothesis': '女性 が 畑 で 踊って い ます 。',
 'label': 'contradiction',
 'premise': '茶色 の ドレス を 着た 女性 が ベンチ に 座って い ます 。'}


In [22]:
import csv
import random
from typing import Iterator

# JSNLI訓練セットから、前提文とラベルごとに仮説文をまとめたdictを作成する
premise2hypotheses = {}

primises = jsnli_dataset["premise"] # 前提文
hypotheses = jsnli_dataset["hypothesis"] # 仮説文
labels = jsnli_dataset["label"] # ラベル

for premise, hypothesis, label in zip(primises, hypotheses, labels):
    if premise not in premise2hypotheses:
        premise2hypotheses[premise] = {
            "entailment": [],
            "neutral": [],
            "contradiction": []
        }
        
    premise2hypotheses[premise][label].append(hypothesis)

In [23]:
def generate_sup_train_example():
    """
    教師ありSimCSEの訓練セットを生成する関数
    Returns:
        dict: 'premise', 'entailment_hypothesis', 'contradiction_hypothesis'をキーとする辞書
    """
    # 結果を格納する辞書を初期化
    dataset_dict = {
        "premise": [],
        "entailment_hypothesis": [],
        "contradiction_hypothesis": []
    }
    
    # JSNLIのデータから三つ組を生成
    for premise, hypotheses in premise2hypotheses.items():
        # 「矛盾」ラベルの仮説文が存在しない場合はスキップ
        if len(hypotheses["contradiction"]) == 0:
            continue
        
        # 「含意」ラベルの仮説文それぞれに対して処理
        for entailment_hypothesis in hypotheses["entailment"]:
            # 「矛盾」ラベルの仮説文をランダムに選択
            contradiction_hypothesis = random.choice(hypotheses["contradiction"])
            
            # 各要素をリストに追加
            dataset_dict["premise"].append(premise)
            dataset_dict["entailment_hypothesis"].append(entailment_hypothesis)
            dataset_dict["contradiction_hypothesis"].append(contradiction_hypothesis)
    
    return dataset_dict

# データセットの作成
try:
    dataset_dict = generate_sup_train_example()
    sup_train_dataset = Dataset.from_dict(dataset_dict)
except Exception as e:
    print(f"エラーが発生しました: {str(e)}")

In [24]:
# 訓練セットの形式と事例数を確認
print(sup_train_dataset)

Dataset({
    features: ['premise', 'entailment_hypothesis', 'contradiction_hypothesis'],
    num_rows: 173438
})


In [25]:
# 訓練セットの内容を確認
pprint(sup_train_dataset[0])
pprint(sup_train_dataset[1])

{'contradiction_hypothesis': '男 が 台所 の テーブル で 本 を 読んで い ます 。',
 'entailment_hypothesis': 'ガレージ に 男 が い ます 。',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'contradiction_hypothesis': '黒人 は デスクトップ コンピューター を 使用 し ます 。',
 'entailment_hypothesis': '人 は 椅子 に 座って い ます 。',
 'premise': 'ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。'}


#### collate関数の準備

In [26]:
from torch import Tensor
from transformers import BatchEncoding

def sup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """訓練セットのミニバッチを作成"""
    premises = []
    hypotheses = []
    
    for example in examples:
        premises.append(example["premise"])
        
        entailment_hypotheis = example["entailment_hypothesis"]
        contradiction_hypothesis = example["contradiction_hypothesis"]
        
        hypotheses.extend(
            [entailment_hypotheis, contradiction_hypothesis]
        )
    # ミニバッチに含まれる前提文と仮説文にトークナイザを適用
    tokenized_premises = tokenizer(
        premises,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )
    tokenized_hypotheses = tokenizer(
        hypotheses,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )
    
    # 前提文と仮説文の類似度行列における正例ペアの位置を示すTensorを作成
    # 行列にi行目の事例（前提文）に対して
    # 2*i列目の要素（仮説文）が正例ペアとなる
    labels = torch.arange(0, 2 * len(premises), 2)
    
    return {
        "tokenized_texts_1": tokenized_premises,
        "tokenized_texts_2": tokenized_hypotheses,
        "labels": labels,
    }

#### モデルの準備

In [27]:
# 教師ありSimCSEモデルを初期化
sup_model = SimCSEModel(base_model_name, mlp_only_train=False)

#### Trainerの準備

In [28]:
# 教師ありSimCSEに訓練のハイパーパラメータを設定
sup_training_args = TrainingArguments(
    output_dir="../model/outputs_sup_simcse",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=5e-5,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=250,
    logging_steps=250,
    save_steps=250,
    save_total_limit=1,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="spearman",
    remove_unused_columns=False,
    report_to="none",  # 外部ツールへのログを無効化
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
# 教師ありSimCSEのTrainerを初期化
sup_trainer = SimCSETrainer(
    model=sup_model,
    args=sup_training_args,
    data_collator=sup_train_collate_fn,
    train_dataset=sup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

#### 訓練の実行

In [30]:
# 教師ありSimCSEの訓練を行う
sup_trainer.train()

TrainOutput(global_step=4065, training_loss=0.8097005076953846, metrics={'train_runtime': 1370.6597, 'train_samples_per_second': 379.608, 'train_steps_per_second': 2.966, 'total_flos': 0.0, 'train_loss': 0.8097005076953846, 'epoch': 3.0})

#### 性能評価

In [31]:
# 検証セットで教師ありSimCSEのモデル評価を行う
sup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.9079980850219727,
 'eval_spearman': 0.8018950489894365,
 'eval_runtime': 10.6323,
 'eval_samples_per_second': 1171.049,
 'eval_steps_per_second': 9.217,
 'epoch': 3.0}

In [32]:
# テストセットで教師ありSimCSEのモデル評価を行う
sup_trainer.evaluate(test_dataset)

{'eval_loss': 2.628929376602173,
 'eval_spearman': 0.8184395733197469,
 'eval_runtime': 1.3935,
 'eval_samples_per_second': 1045.534,
 'eval_steps_per_second': 8.611,
 'epoch': 3.0}

### 最近傍探索ライブラリFaissを使った検索

#### Faissを利用した最近傍探索の実装

In [33]:
# !pip install faiss-cpu==1.7.4

In [34]:
# !pip install numpy

#### データセットの読み込みと前処理

In [35]:
from datasets import load_dataset

paragraph_dataset = load_dataset(
    "llm-book/jawiki-paragraphs", split="train"
)

Using custom data configuration default
Reusing dataset jawiki-paragraphs (/root/.cache/huggingface/datasets/llm-book___jawiki-paragraphs/default/1.0.0/0f2d7acd99ad7ae0615fd07442dbd1654d37c5d60a39fc720efe28acff3f86f8)


In [36]:
# 段落データ形式と事例数を確認
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 9668476
})


In [5]:
from pprint import pprint

# 段落データの内容を確認
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '5-89167474-1',
 'pageid': 5,
 'paragraph_index': 1,
 'revid': 89167474,
 'section': '語源',
 'text': '英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" '
         'も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" '
         'のように唱えられていた。また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。"&" '
         'はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は "X, Y, '
         'Z, and per se and" という形になった。この最後のフレーズが繰り返されるうちに "ampersand" '
         'と訛っていき、この言葉は1837年までには英語の一般的な語法となった。',
 'title': 'アンパサンド'}


In [6]:
# 段落データのうち、各記事の最初の段落のみを使う
paragraph_dataset = paragraph_dataset.filter(
    lambda example: example["paragraph_index"] == 0
)

100%|██████████| 9669/9669 [01:29<00:00, 107.46ba/s]


#### トークナイザとモデルの準備

In [8]:
# !pip install mecab-python3

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 7.8 MB/s eta 0:00:00a 0:00:01
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.10/dist-packages/pip/_v

In [10]:
# !pip install transformers[ja,sentencepice]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 10.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 33.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 23.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556721 sha256=463b39e4a7d2e5a1ee1e993f15b0e8a75aab651ef3a4095e62cd9946f240fff7
  Stored in directory: /tmp/pip-ephem-wheel-cache-2vj8u2b7/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7425 sha256=57ad393d8dfdfb5afff7d3c562756db80d26982fbb4067ba3f4033bb4030523e
  Stored in directory: /tmp/pip-ephem-wheel-cache-2vj8u2b7/wheels/7a/72/72/1f3d654c345ea69d5d51b531c90daf

In [11]:
from transformers import AutoModel, AutoTokenizer

model_name = "llm-book/bert-base-japanese-v3-unsup-simcse-jawiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

In [13]:
# 読み込んだモデルをGPUのメモリに移動
device = "cuda:0"
encoder = encoder.to(device)

#### モデルによる埋め込みの計算

In [16]:
import numpy as np
import torch
import torch.nn.functional as F

def embed_texts(texts: list[str]) -> np.ndarray:
    """SimCSEのモデルを用いてテキストの埋め込みを計算"""
    # テキストにトークナイザを適用
    tokenized_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)
    
    # トークナイズされたテキストをベクトルに変換
    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded_texts = encoder(
                **tokenized_texts
            ).last_hidden_state[:, 0]
            
    # ベクトルをNuPyのarrayに変換
    emb = encoded_texts.cpu().numpy().astype(np.float32)
    # ベクトルのノルムが1になるように正規化
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
    return emb

In [17]:
# 段落データのすべての事例に埋め込みを付与する
paragraph_dataset = paragraph_dataset.map(
    lambda examples: {
        "embeddings": list(embed_texts(examples["text"]))
    },
    batched=True,
)

100%|██████████| 1340/1340 [21:01<00:00,  1.06ba/s]


In [18]:
# 埋め込みを付与した段落データの形式と事例数を確認
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})


In [19]:
# 埋め込みを計算した段落データの内容を確認
pprint(paragraph_dataset[0])

{'embeddings': [0.042540159076452255,
                -0.04196695238351822,
                -0.032314036041498184,
                0.018254535272717476,
                -0.06702621281147003,
                -0.06091688573360443,
                -0.05339366942644119,
                0.00584664149209857,
                0.005609563086181879,
                0.0004761741147376597,
                0.0544019490480423,
                -0.03015177696943283,
                -0.01541396789252758,
                -0.09760468453168869,
                0.031523678451776505,
                0.0070890034548938274,
                0.004207753110677004,
                -0.01840118132531643,
                -0.07030576467514038,
                0.00973212718963623,
                0.006157499272376299,
                -0.03276313096284866,
                -0.00840569194406271,
                -0.023154262453317642,
                0.051235586404800415,
                0.0434039942920208,
              

In [21]:
# 埋め込みを付与した段落データをディスクに保存
paragraph_dataset.save_to_disk(
    "../model/outputs_unsup_simcse/embedded_paragraphs"
)

#### Faissによる最近傍探索を試す

In [32]:
import faiss

# ベクトルの次元数をエンコーダの設定値から取り出す
emb_dim = encoder.config.hidden_size
# ベクトルの次元数を指定して空のFaissインデックスを作成する
index = faiss.IndexFlatIP(emb_dim)
# 段落データの"embeddings"フィールドのベクトルからFaissインデックスを構築する
paragraph_dataset.add_faiss_index("embeddings", custom_index=index)

ImportError: You must install Faiss to use FaissIndex. To do so you can run `conda install -c pytorch faiss-cpu` or `conda install -c pytorch faiss-gpu`. A community supported package is also available on pypi: `pip install faiss-cpu` or `pip install faiss-gpu`. Note that pip may not have the latest version of FAISS, and thus, some of the latest features and bug fixes may not be available.